### naver 지식인에서 '' 검색
* 질문제목, 답변수 확인
* 10페이지(약 500개 답변)
* 답변 크롤링
* 답변 토큰화(okt로 명사만 추출)
* 워드크라우드

In [11]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


import re  # 정규표현식
import pandas as pd
from tqdm.notebook import tqdm
import time

from wordcloud import WordCloud, STOPWORDS
from PIL import Image  # 그림 불러오는 패키지
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


#NLTK Stopwords
from nltk.corpus import stopwords

In [157]:
# .. 검색조건 초기화
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
base_url = "https://kin.naver.com/search/list.naver?query="

# 검색 페이지 수 
p = [str(i+1) for i in range(11)]
   
# 리턴 리스트
line = []

In [ ]:
for j in tqdm(range(len(p))):  # page loop
#for j in[0]:  # page loop

    # 메인 홈페이지
    url = f"{base_url}{quote('아내+결혼기념일+선물')}&page={p[j]}"
    driver.get(url)    
    time.sleep(2)

    # 페이지 소스
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print(soup)
    
    # .. 질문 리스트
    stn = soup.select('ul.basic1 > li')
    print(f'{p[j]} 페이지 질문리스트 개수: {len(stn)}')

    #for i in [0]: # 페이지당 각 질문 리스트 loop
    for i in range(len(stn)):

        # .. 질문 서브 페이지 접속
        # 서브 링크 추출
        sub_link = stn[i].select_one('a')['href']
        #print(sub_link)        

        # 서브 링크 접속
        driver.get(sub_link)
        time.sleep(2)

        # 현재버튼,전체버튼
        num_click = (int(driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text)-1) // 5
        for k in range(num_click):
            driver.find_element(by=By.ID, value='nextPageButton').click()
            time.sleep(2)

        # 서브 링크 소스 
        html_sub = driver.page_source
        soup_sub = BeautifulSoup(html_sub, 'html.parser')
        #print(soup_sub)

        # 질문제목
        title = soup_sub.select_one('div.title').text.strip()
        print(title)
    
        # 답변 개수
        numq = soup_sub.select_one('em._answerCount').text.strip()
        print("답변개수: ", numq)
    
        # 답변 개수별 각 문장 추출
        inners = soup_sub.select('div.answer-content__item._contentWrap._answer')
        #print(len(inner_answ))
        for inner in inners:
            texts = inner.select('p.se-text-paragraph')         
            for text in texts:
                if text == " " or text == '\u200b': continue
                line.append(text)
                #print(text.text)
        driver.back()
        time.sleep(2)